## **Connect to Google Drive**

In [30]:
!pip install gdown

# 64 x 64
!gdown --folder 'https://drive.google.com/drive/folders/19MU_bHOWIPvCrcvHTBzHkvLbAFWt8ZOA'

%cd /kaggle/working/64_64_gray_invert_dilate
%ls

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Retrieving folder list
Processing file 1tuTRtM3R0DXhFwg3cTcXlTeOPYL0DuJt covid.npy
Processing file 1ZovHzOdPABmywo9ECaE97g8CDyz2CQ8u hb.npy
Processing file 1ZvfgjhuB0skNHcxkrIv5UGts_Zy5Hbih mi.npy
Processing file 1HW1u5MEhs5sg0oEv5zoffQ2zpD55_6jp normal.npy
Processing file 1bT7GDNlIblHiHlZTa2rYWdfUSGYLHeum pmi.npy
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1tuTRtM3R0DXhFwg3cTcXlTeOPYL0DuJt
To: /kaggle/working/64_64_gray_invert_dilate/64_64_gray_invert_dilate/covid.npy
100%|███████████████████████████████████████| 13.3M/13.3M [00:00<00:00, 192MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZovHzOdPABmywo9ECaE97g8CDyz2CQ8u
To: /kaggle/working/64_64_gray_invert_dilate/64_64_gra

## **Load Data**

In [31]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

In [32]:
seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

In [33]:
def get_mini_batch(input_X, label, batch_size):
    for i in range(0, len(input_X), batch_size):
        yield input_X[i: i + batch_size], label[i: i + batch_size]

def get_mini(input_X, batch_size):
    for i in range(0, len(input_X), batch_size):
        yield input_X[i: i + batch_size]

In [34]:
def load_images_from_npy(divide=13):
    covid_images = np.load('./covid.npy', allow_pickle=True)
    hb_images = np.load('./hb.npy', allow_pickle=True)
    mi_images = np.load('./mi.npy', allow_pickle=True)
    normal_images = np.load('./normal.npy', allow_pickle=True)
    pmi_images = np.load('./pmi.npy', allow_pickle=True)


    # covid - 0
    # hb - 1
    # mi - 2
    # normal - 3
    # pmi - 4

    # fill labels 0 for covid, 1 for hb, 2 for mi, 3 for normal, 4 for pmi
    labels = [0] * (len(covid_images) // divide)
    labels.extend([1] * (len(hb_images) // divide))
    labels.extend([2] * (len(mi_images) // divide))
    labels.extend([3] * (len(normal_images) // divide))
    labels.extend([4] * (len(pmi_images) // divide))

    images = np.concatenate((covid_images, hb_images, mi_images, normal_images, pmi_images), axis=0)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels


def take_12_leads(data, labels):
    images = []
    new_labels = []

    for X, y in get_mini_batch(data, labels, 13):
        X = np.delete(X, 4, axis=0)
        y = np.delete(y, 4, axis=0)
        images.extend(X)
        new_labels.extend(y)

    images = np.array(images)
    new_labels = np.array(new_labels)
    
    return images, new_labels

def combine_leads_of_image(data, exclude_last=False):
  
    # store each 13 lead images in a list of 13*28*28 arrays
    images = []

    for X in get_mini(data, 13):
        if exclude_last:
            X = np.delete(X, 4, axis=0)
        images.append(X)

    images = np.array(images)
    return images


def stack_images(images):
    stacked_images = np.empty((images.shape[0],images.shape[1]*images.shape[2],images.shape[3]))

    for i in range(images.shape[0]):
        stacked_parts = np.vstack(images[i])
        stacked_images[i] = stacked_parts
    
    return stacked_images

In [35]:
''' 
divide = 1 for 35698 labels
divide = 13 for 2746 labels
'''
images, labels = load_images_from_npy(divide=13)
print(images.shape, labels.shape, images[0].shape)

'''combine 13 leads'''
images = combine_leads_of_image(images, exclude_last=False)
print(images.shape)

(35698, 64, 64) (2746,) (64, 64)
(2746, 13, 64, 64)


In [36]:
# from PIL import Image, ImageOps, ImageFilter
# from IPython.display import display

# def build_image_from_array(img_arr):
#     image = Image.fromarray(img_arr).convert('1')
#     display(image)

# build_image_from_array(images[0][4])

In [37]:
def train_val_split(images, labels, split_factor_train=0.8, split_factor_test=0.9):
    random_indices = np.random.choice(len(images), len(images), replace=False)
    training_data_full = images[random_indices]
    training_label_full = labels[random_indices]

    split_index_train = int(split_factor_train * len(training_data_full))
    split_index_test = int(split_factor_test * len(training_data_full))

    training_data = training_data_full[:split_index_train]
    training_label = training_label_full[:split_index_train]

    validation_data = training_data_full[split_index_train : split_index_test]
    validation_label = training_label_full[split_index_train : split_index_test]

    test_data = training_data_full[split_index_test :]
    test_label = training_label_full[split_index_test :]

    return training_data, training_label, validation_data, validation_label, test_data, test_label

## **Build Model**

In [38]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler

## **CNN_with_Attention**

In [39]:
class ProjectorBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ProjectorBlock, self).__init__()
        self.op = nn.Conv2d(in_channels=in_features, out_channels=out_features,
            kernel_size=1, padding=0, bias=False)

    def forward(self, x):
        return self.op(x)


class SpatialAttn(nn.Module):
    def __init__(self, in_features, normalize_attn=True):
        super(SpatialAttn, self).__init__()
        self.normalize_attn = normalize_attn
        self.op = nn.Conv2d(in_channels=in_features, out_channels=1,
            kernel_size=1, padding=0, bias=False)

    def forward(self, l, g):
        N, C, H, W = l.size()
        c = self.op(l+g) # (batch_size,1,H,W)
        if self.normalize_attn:
            a = F.softmax(c.view(N,1,-1), dim=2).view(N,1,H,W)
        else:
            a = torch.sigmoid(c)
        g = torch.mul(a.expand_as(l), l)
        if self.normalize_attn:
            g = g.view(N,C,-1).sum(dim=2) # (batch_size,C)
        else:
            g = F.adaptive_avg_pool2d(g, (1,1)).view(N,C)
        return c.view(N,1,H,W), g

    

class AttnVGG(nn.Module):
    def __init__(self, sample_size, num_classes, attention=True, normalize_attn=True, init_weights=True):
        super(AttnVGG, self).__init__()
        # conv blocks
        self.conv1 = self._make_layer(13, 64, 2)
        self.conv2 = self._make_layer(64, 128, 2)
        self.conv3 = self._make_layer(128, 256, 3)
        self.conv4 = self._make_layer(256, 512, 3)
        self.conv5 = self._make_layer(512, 512, 3)
        self.conv6 = self._make_layer(512, 512, 2, pool=True)
        self.dense = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=int(sample_size/32), padding=0, bias=True)
        # attention blocks
        self.attention = attention
        if self.attention:
            self.projector = ProjectorBlock(256, 512)
            self.attn1 = SpatialAttn(in_features=512, normalize_attn=normalize_attn)
            self.attn2 = SpatialAttn(in_features=512, normalize_attn=normalize_attn)
            self.attn3 = SpatialAttn(in_features=512, normalize_attn=normalize_attn)
        # final classification layer
        if self.attention:
            self.classify = nn.Linear(in_features=512*3, out_features=num_classes, bias=True)
        else:
            self.classify = nn.Linear(in_features=512, out_features=num_classes, bias=True)
        # if init_weights:
        #     self._initialize_weights()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        l1 = self.conv3(x)
        x = F.max_pool2d(l1, kernel_size=2, stride=2, padding=0)
        l2 = self.conv4(x)
        x = F.max_pool2d(l2, kernel_size=2, stride=2, padding=0)
        l3 = self.conv5(x)
        x = F.max_pool2d(l3, kernel_size=2, stride=2, padding=0)
        x = self.conv6(x)
        g = self.dense(x) # batch_sizex512x1x1

        # attention
        if self.attention:
            c1, g1 = self.attn1(self.projector(l1), g)
            c2, g2 = self.attn2(l2, g)
            c3, g3 = self.attn3(l3, g)
            g = torch.cat((g1,g2,g3), dim=1) # batch_sizex3C
            # classification layer
            x = self.classify(g) # batch_sizexnum_classes
        else:
            c1, c2, c3 = None, None, None
            x = self.classify(torch.squeeze(g))
        
        return x

    def _make_layer(self, in_features, out_features, blocks, pool=False):
        layers = []
        for i in range(blocks):
            conv2d = nn.Conv2d(in_channels=in_features, out_channels=out_features, kernel_size=3, padding=1, bias=False)
            layers += [conv2d, nn.BatchNorm2d(out_features), nn.ReLU(inplace=True)]
            in_features = out_features
            if pool:
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        return nn.Sequential(*layers)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

## **LSTM_with_Attention**

In [40]:
class TemporalAttn(nn.Module):
    def __init__(self, hidden_size):
        super(TemporalAttn, self).__init__()
        self.hidden_size = hidden_size
        self.fc1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.fc2 = nn.Linear(self.hidden_size*2, self.hidden_size, bias=False)

    def forward(self, hidden_states):
        score_first_part = self.fc1(hidden_states)
        h_t = hidden_states[:,-1,:]
        score = torch.bmm(score_first_part, h_t.unsqueeze(2)).squeeze(2)
        attention_weights = F.softmax(score, dim=1)
        context_vector = torch.bmm(hidden_states.permute(0,2,1), attention_weights.unsqueeze(2)).squeeze(2)
        pre_activation = torch.cat((context_vector, h_t), dim=1)
        attention_vector = self.fc2(pre_activation)
        attention_vector = torch.tanh(attention_vector)

        return attention_vector, attention_weights

    
class AttnLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(AttnLSTM, self).__init__()
        self.flatten = nn.Flatten(start_dim=1)
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True)
        self.attn = TemporalAttn(hidden_size=hidden_size)
        self.fc = nn.Linear(hidden_size, 5)

    def forward(self, x):
        x, (h_n, c_n) = self.lstm(x)
        x, weights = self.attn(x)
        x = self.fc(x)
        return x


## **Time_Distributed_CNN**

In [41]:
class TimeDistributed(nn.Module):
    def __init__(self, module, time_steps, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        
        self.layers = nn.ModuleList([module for i in range(time_steps)])
        
        self.time_steps = time_steps
        self.batch_first = batch_first

    def forward(self, x):

        batch_size, time_steps, channels, H, W = x.size()
        output = torch.tensor([]).to("cuda:0")
        
        for i in range(time_steps):
            output_t = self.layers[i](x[:, i, :, :, :])
            output_t  = output_t.unsqueeze(1)
            output = torch.cat((output, output_t), 1)
        
        return output

class Time_Distributed_CNN_Network(nn.Module):
    def __init__(self):
        super(Time_Distributed_CNN_Network, self).__init__()

        self.time_steps = 8
        self.extraction_layers = nn.Sequential(
            TimeDistributed(module=nn.LazyConv2d(out_channels=64, kernel_size=3, padding=1), 
                            time_steps=self.time_steps),
            nn.ReLU(),
            TimeDistributed(module=nn.MaxPool2d(kernel_size=2, stride=2), time_steps=self.time_steps),
            
            TimeDistributed(module=nn.LazyConv2d(out_channels=128, kernel_size=3, padding=1), 
                            time_steps=self.time_steps),
            nn.ReLU(),
            TimeDistributed(module=nn.MaxPool2d(kernel_size=2, stride=2), time_steps=self.time_steps),
        )
        
        
        self.classification_layers = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(5),
            nn.LogSoftmax(dim=1)
        )


    def forward(self, input_X):
        
        for layer in self.extraction_layers:
            input_X = layer(input_X)
    
        for layer in self.classification_layers:
            input_X = layer(input_X)

        return input_X

## **CNN & LSTM_Attention Hybrid**

In [42]:
class HybridNetwork(nn.Module):
    def __init__(self):
        super(HybridNetwork, self).__init__()

        self.extraction_layers = nn.Sequential(
            nn.LazyConv2d(out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.LazyConv2d(out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Flatten(start_dim=1, end_dim=2),
        )
        
#         self.lstm = nn.GRU(input_size=16, hidden_size=10, num_layers=1, batch_first=True, 
#                                 dropout=0.25, bidirectional=False)
        self.attn = AttnLSTM(input_size=16, hidden_size=128, num_layers=1)
        
        self.classification_layers = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(5),
            nn.LogSoftmax(dim=1)
        )


    def forward(self, input_X):
        
        for layer in self.extraction_layers:
            input_X = layer(input_X)
        
#         input_X, (h, c) = self.lstm(input_X) #lstm
#         input_X, h = self.lstm(input_X) #gru
        
#         print('before attn', input_X.shape)
        input_X = self.attn(input_X)
        
        for layer in self.classification_layers:
            input_X = layer(input_X)

        return input_X

## **CNN 2D**

In [43]:
class CNNNetwork(nn.Module):
    def __init__(self):
        super(CNNNetwork, self).__init__()

        self.extraction_layers = nn.Sequential(
            nn.LazyConv2d(out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.LazyConv2d(out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classification_layers = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(5),
            nn.LogSoftmax(dim=1)
        )


    def forward(self, input_X):
        
        for layer in self.extraction_layers:
            input_X = layer(input_X)
        
        for layer in self.classification_layers:
            input_X = layer(input_X)

        return input_X

## **CNN 3D**

In [44]:
class CNN3DNetwork(nn.Module):
    def __init__(self):
        super(CNN3DNetwork, self).__init__()

        
        self.layers = nn.Sequential(
            nn.LazyConv3d(out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.LazyConv3d(out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),
            
            nn.LazyConv3d(out_channels=256, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),
            
            nn.LazyConv3d(out_channels=512, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),
            
            nn.LazyConv3d(out_channels=512, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Flatten(start_dim=1),

            nn.LazyLinear(128),
            nn.ReLU(),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(5),
            nn.LogSoftmax(dim=1)
        )


    def forward(self, input_X):

      for layer in self.layers:
        input_X = layer(input_X)
        # print(input_X.shape)

      return input_X

In [45]:
class Model:
    def __init__(self):
#         self.network = HybridNetwork()
#         self.network = CNNNetwork()
#         self.network = Time_Distributed_CNN_Network()
#         self.network = AttnLSTM(input_size=64, hidden_size=128, num_layers=1)
#         self.network = AttnVGG(sample_size=64, num_classes=5)
        self.network = CNN3DNetwork()
        
        self.num_epochs = 20
        self.batch_size = 16
        self.grad_clipping = 10.0
        self.optimizer_type = 'adamax'
        self.lr = 0.001
        self.num_output_units = 5

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.network.to(self.device)    

        print('Device:', self.device)  

        self.init_optimizer()
        
    def get_network(self):
        return self.network

    def init_optimizer(self):
        parameters = [p for p in self.network.parameters() if p.requires_grad]
        if self.optimizer_type == 'sgd':
            self.optimizer = optim.SGD(parameters, self.lr,
                                        momentum=0.4,
                                        weight_decay=0)
        elif self.optimizer_type == 'adamax':
            self.optimizer = optim.Adamax(parameters,
                                        lr=self.lr,
                                        weight_decay=0)
        else:
            raise RuntimeError('Unsupported optimizer: %s' %
                                self.optimizer_type)
        self.scheduler = lr_scheduler.MultiStepLR(self.optimizer, milestones=[10, 15], gamma=0.5)


    def normalize(self, X):
        # apply standard normalization on x. mean = 0, std = 1
        X = (X - np.mean(X)) / np.std(X)

        return X


    def reshape_image(self, input_X, label):
        
        '''choose one of the below according to the requirement'''
        # for cnn, attnvgg, hybrid
#         X_full = input_X.reshape(input_X.shape[0], input_X.shape[1], input_X.shape[2], input_X.shape[3])
        
        # for cnn3D
        X_full = input_X.reshape(input_X.shape[0], 1, input_X.shape[1], input_X.shape[2], input_X.shape[3])
        
        # for attn lstm
#         X_full = input_X.reshape(input_X.shape[0], input_X.shape[1]*input_X.shape[2], input_X.shape[3])
        
        # time distributed cnn
#         X_full = input_X.reshape(input_X.shape[0], 8, 104, 64)
#         X_full = np.expand_dims(X_full, axis=2)
        
        
        y_full = np.eye(self.num_output_units)[label.astype(int)]

        return X_full, y_full

    def report_num_trainable_parameters(self):
        num_parameters = 0
        for p in self.network.parameters():
            if p.requires_grad:
                sz = list(p.size())

                num_parameters += np.prod(sz)
        print('Number of parameters: ', num_parameters)


    def predict(self, test_data, test_label):
        self.network.eval()

        X = test_data
        y = test_label

        X = self.normalize(X)
        cnt = 0

        for X_batch, y_batch in tqdm(get_mini_batch(input_X=X, label=y, batch_size=self.batch_size), desc="Testing"):

            # reshape to appropriate format
            X, y_true = self.reshape_image(X_batch, y_batch)

            # convert to float32 and convert to torch tensor
            X = X.astype(np.float32)
            y_true = y_true.astype(np.float32)
            X = torch.from_numpy(X)
            y_true = torch.from_numpy(y_true)
            
            X = X.to(self.device)
            y_true = y_true.to(self.device)

            pred_proba = self.network(X)

            y_pred = torch.argmax(pred_proba, dim=1)
            y_true = torch.argmax(y_true, dim=1)
            

            cnt += torch.sum(y_pred == y_true).item()

        print(f'accuracy: {cnt/test_data.shape[0]}')


    def evaluate(self, validation_data, validation_label):
        self.network.eval()

        X = validation_data
        y = validation_label

        X = self.normalize(X)
        cnt = 0

        for X_batch, y_batch in tqdm(get_mini_batch(input_X=X, label=y, batch_size=self.batch_size), desc="Evaluating"):

            # reshape to appropriate format
            X, y_true = self.reshape_image(X_batch, y_batch)

            # convert to float32 and convert to torch tensor
            X = X.astype(np.float32)
            y_true = y_true.astype(np.float32)
            X = torch.from_numpy(X)
            y_true = torch.from_numpy(y_true)

            X = X.to(self.device)
            y_true = y_true.to(self.device)

            pred_proba = self.network(X)

            y_pred = torch.argmax(pred_proba, dim=1)
            y_true = torch.argmax(y_true, dim=1)

            cnt += torch.sum(y_pred == y_true).item()

        print(f'accuracy: {cnt/validation_data.shape[0]}')


    def train(self, input_X, label):
        self.network.train()
        
        self.updates = 0
        iter_cnt, num_iter = 0, (len(input_X) + self.batch_size - 1) // self.batch_size

        for X_batch, y_batch in tqdm(get_mini_batch(input_X=input_X, label=label, batch_size=self.batch_size), desc="Training"):

            # reshape to appropriate format
            X, y_true = self.reshape_image(X_batch, y_batch)

            # convert to float32 and convert to torch tensor
            X = X.astype(np.float32)
            y_true = y_true.astype(np.float32)
            X = torch.from_numpy(X)
            y_true = torch.from_numpy(y_true)
            
            X = X.to(self.device)
            y_true = y_true.to(self.device)

            pred_proba = self.network(X)

            loss = F.cross_entropy(pred_proba, y_true)
            self.optimizer.zero_grad()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(self.network.parameters(), self.grad_clipping)

            # Update parameters
            self.optimizer.step()

            self.updates += 1
            iter_cnt += 1

        
        print('Iter: %d/%d, Loss: %f' % (iter_cnt, num_iter, loss.item()))
        self.scheduler.step()
        print('LR:', self.scheduler.get_last_lr()[0])            


    def fit(self, training_data, training_label, validation_data, validation_label):

        X = training_data
        y = training_label

        X = self.normalize(X)

        for epoch in range(self.num_epochs):
            self.train(input_X=X, label=y)
            print("Epoch: ", epoch)

            self.evaluate(validation_data, validation_label)

In [46]:
def clear_cuda_memory():
    import gc
    # del model
    gc.collect()
    torch.cuda.empty_cache()

In [47]:
training_data, training_label, validation_data, validation_label, \
    test_data, test_label = train_val_split(images=images, labels=labels, 
                                split_factor_train=0.8, split_factor_test=0.9)

print(training_data.shape, training_label.shape, validation_data.shape, validation_label.shape, test_data.shape, test_label.shape)

(2196, 13, 64, 64) (2196,) (275, 13, 64, 64) (275,) (275, 13, 64, 64) (275,)


In [48]:
def main():
    
    model = Model()
    model.evaluate(validation_data, validation_label)
    
    model.fit(training_data, training_label, validation_data, validation_label)
    
    # save model
    # torch.save(model, '/kaggle/working/checkpoint_cnn1.pt')
    
    torch.save(model.get_network().state_dict(), '/kaggle/working/checkpoint_CNN3D.pt')


clear_cuda_memory()    
main()

Device: cuda


Evaluating: 18it [00:00, 31.29it/s]


accuracy: 0.09090909090909091


Training: 138it [00:10, 12.70it/s]


Iter: 138/138, Loss: 0.153159
LR: 0.001
Epoch:  0


...

accuracy: 0.9854545454545455


Training: 138it [00:11, 12.46it/s]


Iter: 138/138, Loss: 0.000000
LR: 0.00025
Epoch:  19


Evaluating: 18it [00:00, 35.84it/s]

accuracy: 0.9854545454545455


## **Testing by Loading Model**

In [53]:
# model = torch.load('/kaggle/working/checkpoint_cnn.pt')
model = Model()
model.get_network().load_state_dict(torch.load('/kaggle/working/checkpoint_CNN3D.pt'))

model.predict(test_data, test_label)

model.predict(validation_data, validation_label)

test_data_concated = np.append(test_data, validation_data, axis = 0)
test_label_concated = np.append(test_label, validation_label, axis = 0)
print(test_data_concated.shape)
print(test_label_concated.shape)

model.predict(test_data_concated, test_label_concated)

Device: cuda


Testing: 18it [00:00, 31.37it/s]


accuracy: 0.9927272727272727


Testing: 18it [00:00, 36.31it/s]


accuracy: 0.9854545454545455
(550, 13, 64, 64)
(550,)


Testing: 35it [00:01, 34.72it/s]

accuracy: 0.9890909090909091


# **End**

In [50]:
# rnn = nn.LSTM(input_size=49, hidden_size=10, num_layers=1, batch_first=True, dropout=0.25, bidirectional=True)
# print(type(rnn))
# input = torch.randn(16, 128, 7, 7)
# print(input.shape)
# flatten = nn.Flatten(start_dim=2, end_dim=3)
# input = flatten(input)
# print(input.shape)
# output, (hn, cn) = rnn(input)
# print(input)

In [51]:
# training_data, training_label, validation_data, validation_label, test_data, test_label = train_val_split(images=images, labels=labels, split_factor=0.8, shuffle=True)
# print(training_data.shape, training_label.shape, validation_data.shape, validation_label.shape, test_data.shape, test_label.shape)
    
# rnn = nn.LSTM(input_size=784, hidden_size=10, num_layers=1, batch_first=True, dropout=0.25, bidirectional=True)
# input = training_data[:16]
# input = input.astype(np.float32)
# input = torch.from_numpy(input)
# print(input.shape)
# flatten = nn.Flatten(start_dim=2, end_dim=3)
# input = flatten(input)
# print(input.shape)
# output, (hn, cn) = rnn(input)

# print(output.shape)

In [52]:
# x = torch.rand(16, 364, 28)
# x = np.array(x).reshape(16,13, 28, 28)
# x = np.expand_dims(x, axis=2)
# x = torch.from_numpy(x)

# model = TimeDistributed(module=nn.LazyConv2d(out_channels=64, kernel_size=3, padding=1), 
#                             time_steps=100)
# output = model(x)

# print(output.shape)